In [1]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4


Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.4 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.5/819.5 kB 4.7 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 7.9 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.3/26.3 MB 40.3 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 23.1 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.0 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 6.4 MB/s eta 0:00:00
    Creating /tmp/pip-build-env-1iaujx1z/overlay/local/bin
    changing mode of /tmp/pip-build-env-1iaujx1z/overlay/local/bin/ninja to 755
    changing mode of /tmp/pip-build-env-1iaujx1z/overlay/lo

In [2]:
import re
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

In [3]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

In [4]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


llama-2-13b-chat.ggmlv3.q5_1.bin:   0%|          | 0.00/9.76G [00:00<?, ?B/s]

In [5]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=1024, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32 # Change this value based on your model and your GPU VRAM pool.
    )


AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [6]:
from google.colab import files
pd= files.upload()

Saving processed_reviews_128 (6).csv to processed_reviews_128 (6).csv


In [7]:
import pandas as pd
import random
df = pd.read_csv('processed_reviews_128 (6).csv')


In [8]:
reviews_df = df[['sentiment', 'cleaned_review']]
reviews_df

,sentiment,cleaned_review
0,positive,mani funni scene peopl dont normal pay attent ...
1,negative,shame shame fine actor joseph fienn would allo...
2,negative,movi pretti predict nuff saidfrom delay kiss s...
3,positive,black mask jet li play bioengin superkil turn ...
4,negative,watch entir movi recogn particip william hurt ...
...,...,...
1995,negative,horribl mish mash predict stori line toebendin...
1996,negative,thoughtless ignor illconceiv careerkil talent ...
1997,positive,saw film ae channel past weekend mysteri okay ...
1998,negative,home weekend bore lack move absolut noth bette...


In [9]:
#cetroid
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances
import numpy as np


In [14]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(reviews_df['cleaned_review'])

In [15]:
positive_indices = reviews_df[reviews_df['sentiment'] == 'positive'].index
negative_indices = reviews_df[reviews_df['sentiment'] == 'negative'].index

In [16]:
def calculate_centroids(df, X, sentiment_column='sentiment'):
    positive_indices = df[df[sentiment_column] == 'positive'].index
    negative_indices = df[df[sentiment_column] == 'negative'].index

    positive_vectors = X[positive_indices]
    negative_vectors = X[negative_indices]

    positive_centroid = np.mean(positive_vectors, axis=0)
    negative_centroid = np.mean(negative_vectors, axis=0)

    return positive_centroid, negative_centroid

In [17]:
positive_centroid, negative_centroid = calculate_centroids(reviews_df, X)

In [18]:
def find_closest_to_centroid(indices, centroid, X):
    centroid = np.asarray(centroid)

    distances = pairwise_distances(X[indices], centroid, metric='euclidean')

    closest_index = np.argmin(distances)

    return indices[closest_index]

In [19]:
closest_positive_review_index = find_closest_to_centroid(positive_indices, positive_centroid, X)
closest_negative_review_index = find_closest_to_centroid(negative_indices, negative_centroid, X)
closest_positive_review = reviews_df.loc[closest_positive_review_index, 'cleaned_review']
closest_negative_review = reviews_df.loc[closest_negative_review_index, 'cleaned_review']

print("Текст ближайшего положительного отзыва:", closest_positive_review)
print("Текст ближайшего отрицательного отзыва:", closest_negative_review)

Текст ближайшего положительного отзыва: watch film tv awesom br br plan watch whilst work howev end watch whole film work good br br actor well know stori line make fact actor make less well known make believ event could occur feel bias toward one charact judgement type charact never seen actor film made even enjoyablebr br would recommend watch
Текст ближайшего отрицательного отзыва: terribl movi one wast time go see someth els movi without doubt one worst movi ever seen life want see good movi dont see made men


In [20]:
df_incorrect = df.loc[df['sentiment'] == 'negative']['cleaned_review'].reset_index(drop=True)
df_correct = df.loc[df['sentiment'] == 'positive']['cleaned_review'].reset_index(drop=True)
df_correct[random.choice(range(len(df_correct)))]

'wow wonder film script nearli perfect appear film written minglun weii hope stori himbr br act sublim reni zhou doggi amaz natur talent xu zhu delight believ jade old traditionalist br br soundtrack effect guid without overwhelm br br movi like made whether hollywood hong kong famili friendli well act well written well direct near perfect gem'

In [21]:
df_incorrect[random.choice(range(len(df_incorrect)))]

'movi terribl almost good almost love music one even terribl sound qualiti poor cinematographi mani actor cant sing danc anthoni rapp actual manag give good perform especi toward end charact marjori drunk ladi enjoy watch br br plot unexpect could funni without terribl sing cheezi piano music admittadli song fantabul pretti catchi good waybr br open hous funni movi watch simpli aw think might good stage music excel actor'

In [ ]:
df_correct_ = 'peopl call comedi watch laughedbr br guess problem first saw br br wasnt old enough understand wasnt meant bebr br taken serious quit scene meantbr br funni care much charact laugh atbr br thembr br suggest watch film next time your fall lovebr br tri take serious think youll find despit sillybr br flaw one move love stori youv ever seen'
df_incorrect_ = 'read review decid rent dvd version br br like classic music want learn bach br br disappoint guess know enough bach music comment enough understand import music play br br mayb would appropri guidanc expert bach music explain film br br realli tri saw whole film hop would abl enjoy least br br see risk'

In [56]:
closest_positive_review =  'Funny movie with relatable moments and great effort from actors and premise. Madonna fumbles through city and puma scenes, but music provides perfect happy ending.'
closest_negative_review = 'Made Men is a terrible movie with no redeeming qualities, according to one reviewer. The plot is nauseating drivel and the acting is shameful, particularly from Joseph Fiennes who should know better. Another reviewer mentions watching the entire movie but does not provide a clear opinion of it. Overall, the reviews are negative with one strongly advising against wasting time on it. Its recommended to watch Pulp Fiction instead which is a great Quentin Tarantino film and much better than this poor parody attempt.'

In [ ]:
#Step 6: Generating the Response

In [57]:
def generate_prompt(df, index):
    return f'''Examples of movie review:
    1. {closest_positive_review}
     ANSWER: positive
    2. {closest_negative_review}
     ANSWER: negative

    Please read and analyze both reviews. Read and correlate the 3rd review to either the 1st or 2nd review based on emotional tone.:
    3th review: "{df['cleaned_review'][index][:128]}"
    '''

END_INDEX = 10

prompts = []
model_r = []
model_responses = []
review_index = 9

while review_index < END_INDEX:
    prompt = generate_prompt(df, review_index)

    prompt_template = f'''SYSTEM:You are a movie review evaluator. Your task is to correlate the 3rd review to either the 1st or 2nd review based on emotional tone

    USER: {prompt}

    ASSISTANT:
    '''

    response = lcpp_llm(prompt=prompt_template, max_tokens=512, temperature=0.5, top_p=0.95,
                      repeat_penalty=1.2, top_k=150,
                      echo=True)

    print(response)
    print(response["choices"][0]["text"])

    response_text = response["choices"][0]["text"]

    matches_ = re.search(r'ASSISTANT:(.*?)(?=Model response|$)', response_text, re.DOTALL)

    if matches_:
        assistant_response = matches_.group(1).strip()
        answer_match = re.search(r'\b(?:1(?:st|\.|rst)?|first|2(?:nd|\.|cond)?|second)\b', assistant_response, re.IGNORECASE)

        # Ensure that only one value is appended for each iteration to `model_responses`
        matched_review_index = answer_match.group(0) if answer_match else None
        model_responses.append(matched_review_index)

        if answer_match:  # Append only when there is a match
            matched_review_index = 1 if answer_match.group().lower() in ['1st', '1.', '1', 'first'] else 2
            prompts.append(prompt_template)
            model_r.append(assistant_response)
        else:  # Append None or placeholders when there's no match
            prompts.append(prompt_template)
            model_r.append(None)  # Or some placeholder that indicates no valid response was found
    else:
        print("Model response does not contain a valid assistant response. Retrying...")
        # Append None or placeholders in this case too
        prompts.append(prompt_template)
        model_r.append(None)
        model_responses.append(None)

    review_index += 1

df_results = pd.DataFrame({
    'Prompt': prompts,
    'Model': model_r,
    'Model Response': model_responses,
})




Llama.generate: prefix-match hit


{'id': 'cmpl-81dfb0b7-7d38-4e66-bfa1-cbc43341e6d4', 'object': 'text_completion', 'created': 1704976769, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': 'SYSTEM:You are a movie review evaluator. Your task is to correlate the 3rd review to either the 1st or 2nd review based on emotional tone\n\n    USER: Examples of movie review:\n    1. Funny movie with relatable moments and great effort from actors and premise. Madonna fumbles through city and puma scenes, but music provides perfect happy ending.\n     ANSWER: positive\n    2. Made Men is a terrible movie with no redeeming qualities, according to one reviewer. The plot is nauseating drivel and the acting is shameful, particularly from Joseph Fiennes who should know better. Another reviewer mentions watching the entire movie but does not provide a clear opinion of it. Overall, the reviews are negative

In [58]:
df_results

,Prompt,Model,Model Response
0,SYSTEM:You are a movie review evaluator. Your ...,"Based on the third review, I would say that it...",second


In [54]:
def generate_prompt_summ(df, index):
    return f'''Rewiews:
    1. {closest_negative_review}
    2. "watch pulp fiction dont see movi movi funni worst parodi movi ever poor attempt parodi filmsbr br cast bad film bad one worst pictur ever madebr br recommend plump fiction prefer origin pulp fiction great quentin tarantino one worst parodi film ever madebr br plump fiction good movi funni dumb vulgar"

    Please read and combine these 2 reviews and write a summary. Limit yourself to 128 tokens per response.  highlight only the main words and DO NOT change their spelling"
    '''

prompt = generate_prompt_summ(df, review_index)

prompt_template = f'''SYSTEM:You are a summary maker.

USER: {prompt}

ASSISTANT:
    '''

response = lcpp_llm(prompt=prompt_template, max_tokens=512, temperature=0.5, top_p=0.95,
                      repeat_penalty=1.2, top_k=150,
                      echo=True)

print(response)
print(response["choices"][0]["text"])

Llama.generate: prefix-match hit


{'id': 'cmpl-9a159d01-f5e2-4161-baec-9cb3bae04cf7', 'object': 'text_completion', 'created': 1704976632, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': 'SYSTEM:You are a summary maker.\n\nUSER: Rewiews:\n    1. Made Men is a terrible movie with no redeeming qualities, according to one reviewer. The plot is nauseating drivel and the acting is shameful, particularly from Joseph Fiennes who should know better. Another reviewer mentions watching the entire movie and recognizing William Hurt, Natascha McElhone, and Desiree Nosbusch, but does not provide a clear opinion of the film. Overall, it seems that the reviews are negative, with one reviewer strongly advising against wasting time on the movie.\n    2. "watch pulp fiction dont see movi movi funni worst parodi movi ever poor attempt parodi filmsbr br cast bad film bad one worst pictur ever madebr br r

In [ ]:
df_results.to_csv('responses50_100.csv', index=False)
files.download('responses50_100.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [59]:
!pip install scikit-learn

true_labels = [1,0,0,1,0,0,1,0,0,0]
predicted_labels = [1,0,1,0,0,1,1,1,0,0]

correct_predictions = sum(1 for true, predicted in zip(true_labels, predicted_labels) if true == predicted)

accuracy = correct_predictions / len(true_labels)

print(f"Точность модели: {accuracy * 100:.2f}%")

Точность модели: 60.00%
